In [2]:
pip install openai ipywidgets pillow python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:
# -*- coding: utf-8 -*-
"""
Visa Assistant with OpenAI Function Calling
"""

import json
import os
import base64
from openai import OpenAI
from IPython.display import display, clear_output
import ipywidgets as widgets
from dotenv import load_dotenv

# --- Configuration ---
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

with open("visa_information.json") as f:
    VISA_DATA = json.load(f)

# --- Country Metadata ---
COUNTRY_DATA = {
    "LBN": {"demonym": "Lebanese", "consulate": "Portuguese"},
    "PRT": {"demonym": "Portuguese", "consulate": "Portuguese"},
    "SGP": {"demonym": "Singaporean", "consulate": "Singaporean"},
    "CYP": {"demonym": "Cypriot", "consulate": "Cypriot"},
    "ISL": {"demonym": "Icelandic", "consulate": "Icelandic"}
}

# --- OpenAI Function Definitions ---
DESTINATION_FUNCTION = {
    "type": "function",
    "function": {
        "name": "extract_destination_code",
        "description": "Extract 3-letter ISO country code from travel destination mention",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_code": {
                    "type": "string",
                    "description": "3-letter ISO country code",
                    "pattern": "^[A-Z]{3}$"
                }
            },
            "required": ["destination_code"]
        }
    }
}

PASSPORT_FUNCTION = {
    "type": "function",
    "function": {
        "name": "extract_passport_code",
        "description": "Extract 3-letter ISO country code from passport nationality",
        "parameters": {
            "type": "object",
            "properties": {
                "passport_code": {
                    "type": "string",
                    "description": "3-letter ISO country code",
                    "pattern": "^[A-Z]{3}$"
                }
            },
            "required": ["passport_code"]
        }
    }
}

# --- Core Visa Logic ---
def get_visa_recommendation(passport_code, destination_code):
    """Returns formatted recommendation based on JSON data"""
    destination = VISA_DATA.get(destination_code.upper())
    
    if not destination:
        return "No visa information available for this destination."
    
    visa_free = destination.get("visaFree", {}).get(passport_code.upper())
    if visa_free:
        return (
            f"{COUNTRY_DATA.get(passport_code, {}).get('demonym', passport_code)} "
            f"passport holders are exempt from visa for travel to {destination_code} "
            f"with a maximum stay of {visa_free['maxStay']} days."
        )
    
    visa_on_arrival = destination.get("visaOnArrival", {}).get(passport_code.upper())
    if visa_on_arrival:
        return (
            f"{COUNTRY_DATA.get(passport_code, {}).get('demonym', passport_code)} "
            f"passport holders can obtain a visa upon arrival in {destination_code} "
            f"with a maximum stay of {visa_on_arrival['maxStay']} days."
        )
    
    return (
        f"{COUNTRY_DATA.get(passport_code, {}).get('demonym', passport_code)} "
        f"passport holders need to apply for a visa in advance at a "
        f"{COUNTRY_DATA.get(destination_code, {}).get('consulate', destination_code)} "
        f"consulate - no visa exemptions or arrivals available."
    )

# --- Enhanced Chat Workflow with Function Calling ---
def visa_chat():
    """Main conversation flow using OpenAI Function Calling"""
    print("AI: Welcome to Visa Assistant! How can I help you?")
    
    # Phase 1: Destination Detection
    dest_code = None
    while not dest_code:
        user_input = input("\nUser: ")
        
        # Extract destination using function calling
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": user_input}],
            tools=[DESTINATION_FUNCTION],
            tool_choice={"type": "function", "function": {"name": "extract_destination_code"}}
        )
        
        if response.choices[0].message.tool_calls:
            args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
            candidate_code = args["destination_code"].upper()
            if candidate_code in VISA_DATA:
                dest_code = candidate_code
                break
                
        print("AI: Could you clarify which country you're planning to visit?")

    # Phase 2: Passport Verification
    print("AI: Thank you. Could you specify your passport nationality?")
    passport_code = None
    
    while not passport_code:
        user_input = input("\nUser: ").strip()
        
        # Handle image upload
        if user_input.lower() == "upload":
            uploader = widgets.FileUpload(accept='image/*')
            display(uploader)
            input("Press Enter after uploading...")
            
            if uploader.value:
                image_data = uploader.value[0].content
                response = client.chat.completions.create(
                    model="gpt-4o",
                    messages=[{
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "Extract 3-letter ISO country code from passport MRZ"},
                            {"type": "image_url", "image_url": {
                                "url": f"data:image/jpeg;base64,{base64.b64encode(image_data).decode('utf-8')}"
                            }}
                        ]
                    }],
                    max_tokens=5
                )
                passport_code = response.choices[0].message.content.strip().upper()
        
        # Text input processing with function calling
        if not passport_code and user_input:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": user_input}],
                tools=[PASSPORT_FUNCTION],
                tool_choice={"type": "function", "function": {"name": "extract_passport_code"}}
            )
            
            if response.choices[0].message.tool_calls:
                args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
                candidate_code = args["passport_code"].upper()
                if candidate_code in COUNTRY_DATA:
                    passport_code = candidate_code
        
        if not passport_code:
            print("AI: Please provide a valid passport country or type 'upload' to submit a passport image")

    # Generate final response
    print("\nAI: " + get_visa_recommendation(passport_code, dest_code))

# --- Run the application ---
if __name__ == "__main__":
    visa_chat()

AI: Welcome to Visa Assistant! How can I help you?



User:  Hello, I'm planning a trip from to Portugal. Can you help me with the visa requirements?


AI: Thank you. Could you specify your passport nationality?



User:  I’m from Singapore



AI: Singaporean passport holders are exempt from visa for travel to PRT with a maximum stay of 30 days.
